In [ ]:
import numpy as np
import pandas as pd
from pandas import HDFStore

import os

import datetime
from datetime import datetime

from plotly.offline import init_notebook_mode
from IPython.display import Image, display


In [ ]:
init_notebook_mode()
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
def load_data(path,condition):
    
    db = HDFStore(path)
    
    ev = db.select('events', where = condition)
#    gr = db.select('gradients', where = condition)
    gr = db.select('details/gradients', where = condition)
    de = db.select('details/delays', where = condition)
    
    db.close()
    
    return ev, gr, de

def print_time():
    print(datetime.now().strftime(format='%H:%M:%S'))
    return



def display_event( plots_dir, ev_id, year ):
    
    route = os.path.join( plots_dir[year], str(ev_id)+".png" )  # Display stored picture
    if os.path.exists(route):
        display(Image(filename=route))
    else:
        print "File not found", route 
    return


In [4]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Init

In [5]:
# Define path to plots
base_dir = 'D://VBox/data/itm_ecuador/visualization/'
plots_dir = {2013 : base_dir+'Plots2013', 2014 : base_dir+'Plots2014'}

# Load events manual classification 
ev_class = pd.read_csv('D://phd/data/labels.csv')

### Load data

In [ ]:
data_file = 'D://phd/data/ie.h5'

# Descriptors calculation

## Global descriptors

| name      | description   |
|:-:|:-:    |
|year       | Year (2013-2014)|
|doy        | Time of year (1-365)|
|prn       | Satellite PRN|
|hh         | Time of day (0. - 24.)|
|min_grad   | Maximum gradient |
|mean_grad   | Maximum gradient |
|std_grad   | Maximum gradient |
|n_stations       | Number of stations |
|n_pairs    | Number of pairs |
|el_deg         | Elevation in degrees |
|gradient       | Maximum gradient |


## Individual curve descriptors

### Delay descriptors



| name  | description  | normalization  |
|:-:|:-:|:-:|
|i_range| Maximum phase delay minus Minimum phase delay||
|n_epochs|Number of epochs ||
|roti_min|Min of scintillation index: a ROTI-like indicator (*) ||
|roti_mean|Mean of scintillation index: a ROTI-like indicator||
|roti_max|Max of scintillation index: a ROTI-like indicator||
|roti_std|Std of scintillation index: a ROTI-like indicator||
|n_outliers_ma|Number of outliers from Moving Average method||
|n_outliers_pf|Number of outliers from Polynomial fit method||

(*) Since iono_delay is proportional to TEC, we will 
use iono delay and obtain a ROTI-like index as (calculated over 
a rolling window):

$$SROT = \frac{\Delta I_{delay}}{\Delta t}$$

$$SROTI = \sqrt{<ROT^2> - <ROT>^2 }$$

### First difference descriptors

**diff**: for each station, array containing the difference between consecutive phase delay values station

| name  | description  | normalization |
|:-:|:-:|:-:|
|diff_max| Max diff value during all the event ||
|diff_std| Std diff value during all the event ||
|diff_outliers| Number of points out of +/- n sigmas || 	 	 	 	 	 	 	 	 	 	

In [6]:
def roti_serie(y):
    z  = np.mean(y*y)
    z -= (np.mean(y))*(np.mean(y))
    return np.sqrt( z )

def roti_df(x, p):
    # Definition of ROTI is:
    #
    # ROT = Delta(TEC) / Delta(t)
    # ROTI = SQRT( <ROT^2> - <ROT>^2 )
    #
    # Since iono_delay is proportional to TEC, we will 
    # use iono delay and obtain a ROTI-like index
    
    x[['delta_t','delta_i']] = x[['tod','i_phase']].diff()
    x = x[x['delta_t'] < p['max_dt']]
    x['rot']  = x['delta_i'] / x['delta_t']
    x['roti'] = 100*x['rot'].rolling( window = p['roti_win'] ).apply(roti_serie)
    
    return pd.Series([x['roti'].min(), x['roti'].mean(), x['roti'].max(), x['roti'].std()],
                     index=['roti_min','roti_mean','roti_max','roti_std'])

def outliers_moving_av(x, p):
    x['diff']    = abs(x['i_phase'] - x['i_phase'].rolling(window=p['mov_av_win']).mean())
    x['diff']    = x['diff'].fillna(20)
    threshold  = x['i_phase'].std()*p['std_threshold']
    n_outliers = len(x[ x['diff'] > threshold ])
    return pd.Series([n_outliers], index=['n_outliers_ma'])

def outliers_poly_fit(x, p):

    polynomial = np.poly1d(np.polyfit( x['tod'], x['i_phase'], p['poly_fit_degree'] ))
    x['diff']  = abs(x['i_phase'] - polynomial(x['tod']))
    threshold  = x['i_phase'].std()*p['std_threshold']
    n_outliers = len(x[x['diff']>threshold])
    
    return pd.Series([n_outliers], index=['n_outliers_pf'])

def first_difference(x, p):
    
    x[['delta_t','delta_i']] = x[['tod','i_phase']].diff()
    x = x[x['delta_t'] < p['max_dt']]
    
    outliers = len(x[abs(x['delta_i']-x['delta_i'].mean()) > p['diff_sigma']*abs(x['delta_i'].std())])
    
    return pd.Series([x['delta_i'].max(), x['delta_i'].std(), outliers],
                     index=['diff_max','diff_std','diff_outliers'])

def delay_desc(x, p):

    i_range  = x['i_phase'].max() - x['i_phase'].min()
    n_epochs = len(x)

    desc = []
    
    desc.append(pd.Series([i_range, n_epochs], index=['i_range', 'n_epochs']))
    desc.append(roti_df(x, p))
    desc.append(outliers_moving_av(x,p))
    desc.append(outliers_poly_fit(x,p))
    desc.append(first_difference(x, p))
    
    return pd.concat(desc)

## Process all events and store individual descriptors

In [ ]:
data_file = 'D://phd/data/ie.h5'
desc_file = 'D://phd/data/desc_layer_1.h5'


params = {'max_dt' : 30.*4,
          'roti_win' : 10,
          'mov_av_win' : 10,
          'diff_threshold' : 3,
          'diff_sigma' : 2,
          'poly_fit_degree':4,
          'std_threshold' : 1,
         }
    


In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
desc_db = HDFStore(desc_file)
try:
    desc_db.remove('station_desc')
except:
    print('No data in store')

step=500
year_size = {2013:7000, 2014:12000}

for year in [2013,2014]:
    for start in range(0,year_size[year],step):
    
        print_time()
        print(year, start)
        
        condition  = 'year = '+str(year)
        condition += ' and event_id >= '+str(start)
        condition += ' and event_id <  '+str(start+step)
        
        _, _, y = load_data(data_file, condition)
        
        y = y[['event_id','year','tod','station','i_phase']]
        y.sort_values(by=['event_id','year','tod','station','tod'], inplace=True)
        y_desc = y.groupby(['event_id','year','station'], as_index=False).apply(delay_desc, params)
        y_desc = y_desc.reset_index()
        desc_db.append('station_desc', y_desc, 
                       index=False, data_columns = y_desc.columns)
desc_db.close()

### Aggregate descriptors

In [48]:
# Load descriptors per station
desc_file = 'D://phd/data/desc_layer_1.h5'
desc_db = HDFStore(desc_file)
d = desc_db.select('station_desc')
d['n_epochs'] /= 1.5*60*2.

# Load events general descriptors
data_file = 'D://phd/data/ie.h5'
data_db = HDFStore(data_file)
events = data_db.select('events')
events['doy'] /= 365.
events['hh'] /= 24.

# Aggregate individual curve descriptors
d_group = d.groupby(['event_id','year'], as_index=False).agg([min,max,'mean','std'])
d_group.columns = ['_'.join(col).strip() for col in d_group.columns.values]
d_group.reset_index(inplace=True)

# create a dataframe with general and curve agg features
df = pd.merge(events, d_group, on=['event_id', 'year'])
df.drop('label',axis=1,inplace=True)
df.dropna(inplace=True)
df = df[df.columns.difference( ['label','label_id'])]

try:
    desc_db.remove('event_desc')
    print('Data removed')
except:
    print('No data in store')
    
desc_db.append('event_desc',
               df, 
               index=False,
               data_columns = df.columns)

desc_db.close()

Data removed


## ToDo: Single delay curve Time series descriptors

The time series descriptors are presented this way:

**station -> time_slot -> descriptors**

So, this is a cube of dimension n_stations x n_time_slots x n_features. For each station / time_slot, it is defined:

| name  | description  | values  |
|:-:|:-:|:-:|
|d_kur| diff kurtosis during the time window||
|d_max  | Maximum diff value during during the time window |   |
|d_ske| diff skewness during the time window||
|d_var| diff variance during the time window||
|d_outliers| Number of diff points out of +/- 2*sigma* during the time window. *sigma* is calculated for all the event  ||
| d_outliers_bin  | Binary for diff values out of +/- 2*sigma* during the time window. *sigma* is calculated for all the event   |   |
|i_range| Maximum phase delay minus Minimum phase delay during the time window||
| n| Number of epochs during the time window||
| n_i_out_ma  | Number of outliers in phase delay with the moving average method |   |
| n_i_out_pf  | Number of outliers in phase delay with the polyfit method   |   |
| nr| Number of epochs / Max number of epochs for the event||
| sroti  | Scintillation index: a ROTI-like indicator: get_sroti(data) |   |
